In [1]:
import pandas as pd
import numpy as np
from numpy import zeros

In [2]:
#sklearn imports
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

/home/tasos/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
#load data
data = pd.read_csv('train_set.csv', sep="\t")

In [31]:
#create set from categories
le = preprocessing.LabelEncoder()
le.fit(data["Category"])
y = le.transform(data["Category"])
set(y)

{0, 1, 2, 3, 4}

In [21]:
#import pipeline
from sklearn.pipeline import make_pipeline
#import model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [19]:
metrics       = ['minkowski','euclidean','manhattan'] 
weights       = ['uniform','distance'] #10.0**np.arange(-5,4)
numNeighbors  = np.arange(3,10)
param_grid    = dict(metric=metrics,weights=weights,n_neighbors=numNeighbors)

In [24]:
grid=GridSearchCV(KNeighborsClassifier(),param_grid=param_grid,n_jobs=-1,cv=10)

In [25]:
grid.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': array([3, 4, 5, 6, 7, 8, 9]), 'metric': ['minkowski', 'euclidean', 'manhattan'], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
grid.best_params_

{'metric': 'minkowski', 'n_neighbors': 6, 'weights': 'distance'}

In [27]:
model=make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier(**grid.best_params_))

In [28]:
#import vectorizer and lsi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

In [32]:
#do vectorization
vectorizer = TfidfVectorizer(max_df=0.5,stop_words='english',use_idf=True)
X = vectorizer.fit_transform(data['Content'])
lsa=TruncatedSVD(n_components=100)
X=lsa.fit_transform(X)
X=preprocessing.Normalizer(copy=False).fit_transform(X)

In [33]:
#run model with 10_fold
predicted= cross_val_predict(model, X, y, cv=10)

In [34]:
predicted

array([1, 0, 0, ..., 2, 0, 4])

In [35]:
#import for ignore the famous error
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [36]:
#inverse predicted
predicted_categories = le.inverse_transform(predicted)
predicted_categories

array(['Film', 'Business', 'Business', ..., 'Football', 'Business',
       'Technology'], dtype=object)

In [37]:
#import for metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [38]:
#get metrics for precision,recall,fscore,support in a list
score=precision_recall_fscore_support(y,predicted,average='macro')
precision=score[0]
recall=score[1]
f1_score=score[2]

In [39]:
#get metrics for accuracy
accuracy=accuracy_score(y,predicted)

In [40]:
#open csv for write results
data = pd.read_csv('EvaluationMetric_10fold.csv',sep='\t')

In [41]:
#write results to csv
data.ix[0,'My Method']=accuracy
data.ix[1,'My Method']=precision
data.ix[2,'My Method']=recall
data.ix[3,'My Method']=f1_score
#save/overwrite csv
data.to_csv('EvaluationMetric_10fold.csv',sep='\t',index=False)

In [42]:
data

,Statistic Measure,Naive Bayes,Random Forest,SVM,KNN,My Method
0,Accuracy,0.939426,0.678787,0.959237,0.954962,0.954019
1,Precision,0.935177,0.648057,0.956933,0.953109,0.950669
2,Recall,0.937824,0.647480,0.957114,0.954454,0.953021
3,F-Measure,0.936162,0.647525,0.957014,0.953720,0.951753
